In [2]:
# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np
import tqdm as tqdm
from time import sleep
import random
import time

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from selenium.webdriver.common.by import By

SCRAPPING INE

In [3]:
url = "https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990"

chrome_options = webdriver.ChromeOptions()

prefs = {
    "download.default_directory": "C:\\Users\\alexc\\Desktop\\GIT\\5.1.ETL-Extraccion\\data",  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)





No encuentro el botón
No encuentro si es 2019
No encuentro provincia
No encuentro el botón


EXTRACCION API

In [43]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [44]:
ListIdcomunidades = list(cod_comunidades.values())

In [62]:
# Definir URLs de los endpoints
base_url_demanda = "https://apidatos.ree.es/es/datos/demanda/evolucion"
base_url_generacion = "https://apidatos.ree.es/es/datos/generacion/estructura-renovables"
headers = {'Accept': 'application/json'}

# Función para extraer datos de un solo año
def obtener_datos(tipo, anio, comunidad_id):
    base_url = base_url_demanda if tipo == 'demanda' else base_url_generacion
    fecha_inicio = f"{anio}-01-01"
    fecha_fin = f"{anio}-12-31"
    params = {'start_date': fecha_inicio,
            'end_date': fecha_fin,
            'time_trunc': 'month',
            'geo_limit': 'ccaa',
            'geo_ids': comunidad_id}
    response = requests.get(base_url, headers=headers, params=params)
    
    # Revisar si la solicitud fue exitosa
    if response.status_code == 200:
        data = response.json()
        valores = data['included'][0]['attributes']['values']
        df = pd.DataFrame(valores)
        df['Tipo'] = tipo
        df['Id_Comunidad']=comunidad_id
        return df
    else:
        print(f"Error {response.status_code} al solicitar {tipo} para {comunidad_id} en el año {anio}")
        return pd.DataFrame()

In [55]:
lista_años = list(range(2019, 2022))

listadf_demanda=[]
listadf_generacion=[]

for id in ListIdcomunidades:
    for año in lista_años:
        datos_demanda = obtener_datos('demanda', año, id)
        datos_generacion = obtener_datos('generacion',año, id)
        listadf_demanda.append(datos_demanda)
        listadf_generacion.append(datos_generacion)


Error 502 al solicitar generacion para 8744 en el año 2019
Error 502 al solicitar generacion para 8744 en el año 2020
Error 502 al solicitar generacion para 8744 en el año 2021


In [58]:
df_demanda=pd.concat(listadf_demanda,ignore_index=True)
df_generacion=pd.concat(listadf_generacion,ignore_index=True)

In [63]:
df_demanda.dtypes

value           float64
percentage        int64
datetime         object
Tipo             object
Id_Comunidad      int64
dtype: object

In [64]:
df_generacion.dtypes

value           float64
percentage      float64
datetime         object
Tipo             object
Id_Comunidad      int64
dtype: object